<a href="https://colab.research.google.com/github/renatojmf/Aspects-Extraction-in-Portuguese/blob/main/Avaliacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install sklearn-crfsuite
! pip install eli5
! pip install unidecode
! pip install stanza

In [ ]:
import pandas as pd     
import numpy as np
from bs4 import BeautifulSoup
import re
import nltk  
nltk.download('punkt')
from nltk.corpus import stopwords # Import the stop word list

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from time import time

In [77]:
data = pd.read_csv('/content/restaurantes.csv',sep=';')

In [ ]:
import stanza as st
import pandas as pd
st.download('pt') # download pt model
nlp = st.Pipeline('pt') # initialize pt neural pipeline

In [13]:
from unidecode import unidecode
import re

words = []

for word in data['text']:
  old_string = unidecode(word)
  new_string = re.sub(r"[^a-zA-Z0-9]"," ",old_string)
  words.append(new_string.lower())

In [78]:
data['text'] = words
data['labels'] =-1
data.head()

,text,aspects,aspectsClass,labels
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food,-1
1,excelente peixe para quem esta localizado no ...,peixe,Food,-1
2,serve lanches e cerveja gelada tem mesinhas f...,cerveja,Drinks,-1
3,muito bom o atendimento,atendimento,Service,-1
4,cardapio variado,Cardápio,Food,-1


In [80]:
data.loc[data["aspectsClass"] == "Service", "labels"] = 0
data.loc[data["aspectsClass"] == "Food", "labels"] = 1
data.loc[data["aspectsClass"] == "Price", "labels"] = 2
data.loc[data["aspectsClass"] == "Ambience", "labels"] = 3
data.loc[data["aspectsClass"] == "Location", "labels"] = 4

In [81]:
df = data[data['labels'] != -1]
df.head()

,text,aspects,aspectsClass,labels
0,hamburgueria artesanal com tempero diferenciad...,tempero,Food,1
1,excelente peixe para quem esta localizado no ...,peixe,Food,1
3,muito bom o atendimento,atendimento,Service,0
4,cardapio variado,Cardápio,Food,1
5,ambiente agradavel,Ambiente,Ambience,3


In [116]:
X_train, X_test, y_train, y_test = train_test_split(df["text"], df["labels"], test_size = 0.3, random_state=42)

In [117]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf_vectorizer = CountVectorizer() # or term frequency
X_train_tf = tf_vectorizer.fit_transform(X_train)
X_test_tf = tf_vectorizer.transform(X_test)

In [119]:
from sklearn.naive_bayes import MultinomialNB
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, y_train)
y_pred = naive_bayes_classifier.predict(X_test_tf)

In [120]:
from sklearn import metrics
t = time()
naive_bayes_classifier.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = naive_bayes_classifier.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')
#print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

train time: 0.003s
test time:  0.000s
accuracy:   0.492
              precision    recall  f1-score   support

     Service       0.38      0.38      0.38        13
        Food       0.69      0.69      0.69        26
       Price       0.00      0.00      0.00         4
    Ambience       0.47      0.47      0.47        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.49        61
   macro avg       0.31      0.31      0.31        61
weighted avg       0.49      0.49      0.49        61

confusion matrix:
[[ 5  4  1  3  0]
 [ 2 18  1  2  3]
 [ 1  1  0  1  1]
 [ 4  3  1  7  0]
 [ 1  0  0  2  0]]
------------------------------


In [115]:
from sklearn.tree import DecisionTreeClassifier
dTree = DecisionTreeClassifier()

t = time()
dTree.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = dTree.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.017s
test time:  0.002s
accuracy:   0.541
              precision    recall  f1-score   support

     Service       0.44      0.54      0.48        13
        Food       0.64      0.62      0.63        26
       Price       0.00      0.00      0.00         4
    Ambience       0.67      0.53      0.59        15
    Location       0.40      0.67      0.50         3

    accuracy                           0.54        61
   macro avg       0.43      0.47      0.44        61
weighted avg       0.55      0.54      0.54        61

confusion matrix:
[[ 7  3  2  1  0]
 [ 4 16  1  3  2]
 [ 1  2  0  0  1]
 [ 4  3  0  8  0]
 [ 0  1  0  0  2]]
------------------------------


In [123]:
from sklearn.svm import SVC
svm = SVC()

t = time()
svm.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = svm.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.020s
test time:  0.002s
accuracy:   0.475
              precision    recall  f1-score   support

     Service       0.67      0.46      0.55        13
        Food       0.48      0.77      0.59        26
       Price       0.00      0.00      0.00         4
    Ambience       0.50      0.20      0.29        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.48        61
   macro avg       0.33      0.29      0.28        61
weighted avg       0.47      0.48      0.44        61

confusion matrix:
[[ 6  5  1  1  0]
 [ 2 20  0  2  2]
 [ 1  2  0  0  1]
 [ 0 12  0  3  0]
 [ 0  3  0  0  0]]
------------------------------


In [124]:
from sklearn.linear_model import LogisticRegression
regression = LogisticRegression()

t = time()
regression.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = regression.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.068s
test time:  0.000s
accuracy:   0.574
              precision    recall  f1-score   support

     Service       0.64      0.54      0.58        13
        Food       0.65      0.77      0.70        26
       Price       0.00      0.00      0.00         4
    Ambience       0.57      0.53      0.55        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.57        61
   macro avg       0.37      0.37      0.37        61
weighted avg       0.55      0.57      0.56        61

confusion matrix:
[[ 7  3  1  2  0]
 [ 2 20  0  2  2]
 [ 1  2  0  0  1]
 [ 1  5  1  8  0]
 [ 0  1  0  2  0]]
------------------------------


In [128]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()

t = time()
mlp.fit(X_train_tf,y_train)
duration = time() - t
print("train time: %0.3fs" % duration)

t = time()
y_pred = mlp.predict(X_test_tf)
test_time = time() - t
print("test time:  %0.3fs" % test_time)

# compute the performance measures
score1 = metrics.accuracy_score(y_test, y_pred)
print("accuracy:   %0.3f" % score1)

print(metrics.classification_report(y_test, y_pred,target_names = ['Service','Food','Price','Ambience','Location']))

print("confusion matrix:")
print(metrics.confusion_matrix(y_test, y_pred))

print('------------------------------')

train time: 0.675s
test time:  0.002s
accuracy:   0.557
              precision    recall  f1-score   support

     Service       0.50      0.46      0.48        13
        Food       0.71      0.77      0.74        26
       Price       0.00      0.00      0.00         4
    Ambience       0.53      0.53      0.53        15
    Location       0.00      0.00      0.00         3

    accuracy                           0.56        61
   macro avg       0.35      0.35      0.35        61
weighted avg       0.54      0.56      0.55        61

confusion matrix:
[[ 6  4  2  1  0]
 [ 1 20  0  3  2]
 [ 1  1  0  1  1]
 [ 3  3  1  8  0]
 [ 1  0  0  2  0]]
------------------------------


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
